In [421]:
"""
Скрипт для обработки форм мониторинга кадровой потребности ОПК августовская форма
"""

'\nСкрипт для обработки форм мониторинга кадровой потребности ОПК августовская форма\n'

In [422]:
# -*- coding: UTF-8 -*-
import pandas as pd
import numpy as np
import tkinter
import sys
import os
from tkinter import *
from tkinter import filedialog
from tkinter import messagebox
from tkinter import ttk
import time
# pd.options.mode.chained_assignment = None  # default='warn'
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None
import copy
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import re
import random
pd.options.display.width = 0

In [423]:
def check_correct_relation(df,first_column,second_column,correct_number):
    """
    Функция для проверки правильности заполнения двух колонок. 
    Создаем словарь где ключеом является значение first_column  а значением second_column. 
    Если для first_column встречаются отличающиеся значения то это определяется как ошибка, при этом выдается номер строки где произошло
    с поправочным коэфициентом correct_number
    """
    _error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ]) # датафрейм с ошибками
    dct_check = {} # создаем словарь для хранения данных  в формате {first_column:second_column}
    
    for idx,row in enumerate(df.itertuples(),correct_number):        
        first_value = row[first_column] # создаем переменные чтобы было легче читать код
        second_value = row[second_column]
        if first_value not in dct_check:
            dct_check.setdefault(first_value,second_value) # если такого ключа нет то создаем пару ключ значение
        else:
            # проверяем название организации
            if dct_check[first_value] != second_value:
                temp_error_df = pd.DataFrame(data=[[f'{name_file}', f'Проверьте правильность написания названия организации в строке {idx} ',
                                                f'Существует другое написание названия организации для указанного ИНН -{first_value}']],
                                         columns=['Название файла', 'Строка или колонка с ошибкой',
                                                  'Описание ошибки'])
                _error_df = pd.concat([_error_df, temp_error_df], axis=0, ignore_index=True)
    
    return _error_df
    

In [424]:
path_data_opk = 'data/Форма август'
path_to_end_folder = 'data'
file_form_data = 'data/Форма сбора данных.xlsx'

In [425]:
# дождусь пока форма не устаканится и потом добавллю эту проверку
lst_mon_col = [f'гр.{i}' for i in range(1,26)] # создаем список для проверки колонок данных по мониторингу
lst_act_col = [f'гр.{i}' for i in range(1,13)] # создаем список для проверки колонок данных по мерам


In [426]:
# Создаем итоговые датафреймы
base_mon_df = pd.DataFrame(columns=lst_mon_col)
base_action_df = pd.DataFrame(columns=lst_act_col)

In [427]:
error_df = pd.DataFrame(columns=['Название файла', 'Строка или колонка с ошибкой', 'Описание ошибки', ]) # датафрейм с ошибками


In [428]:
for file in os.listdir(path_data_opk):
    if not file.startswith('~$') and file.endswith('.xls'):
        name_file = file.split('.xls')[0]
        temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                            'Файл с расширением XLS (СТАРЫЙ ФОРМАТ EXCEL)!!! ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                     columns=['Название файла', 'Строка или колонка с ошибкой',
                                              'Описание ошибки'])
        error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
        continue
    if not file.startswith('~$') and file.endswith('.xlsx'):
        name_file = file.split('.xlsx')[0]
        print(name_file)
        # Проверяем наличие листов с названиями Форма 1 и Форма 2
        wb_1 = openpyxl.load_workbook(f'{path_data_opk}/{file}',read_only=True)
        if not {'Форма по мониторингу','Форма по принимаемым мерам'}.issubset(set(wb_1.sheetnames)):
            temp_error_df = pd.DataFrame(data=[[f'{name_file}', '',
                                                'Проверьте наличие листов с названием Форма по мониторингу и Форма по принимаемым мерам! Не должно быть пробелов в начале и в конце названия ДАННЫЕ ФАЙЛА НЕ ОБРАБОТАНЫ !!! ']],
                                         columns=['Название файла', 'Строка или колонка с ошибкой',
                                                  'Описание ошибки'])
            error_df = pd.concat([error_df, temp_error_df], axis=0, ignore_index=True)
            continue
    wb_1.close() # закрываем 
    
    # считываем датафреймы
    df_mon = pd.read_excel(f'{path_data_opk}/{file}',sheet_name='Форма по мониторингу',dtype=str,skiprows=3,header=None) # лист с данными мониторинга
    df_action = pd.read_excel(f'{path_data_opk}/{file}',sheet_name='Форма по принимаемым мерам',dtype=str,skiprows=3,header=None) # лист с данными по принимаемым мерам
    
    # Присваиваем названия колонкам
    df_mon.columns = lst_mon_col
    df_action.columns = lst_act_col

    
    # удаляем полностью пустые строки
    df_mon.dropna(thresh=7,inplace=True)
    df_action.dropna(thresh=7,inplace=True)
    
    # Проверяем совпадение ИНН.
    """
    Проверка на ошибки
    """
    
    # добавляем название файла
    df_mon['Название файла'] =name_file
    df_action['Название файла'] = name_file
    
    # добавляем в базовые датафреймы
    base_mon_df = pd.concat([base_mon_df,df_mon],ignore_index=True)
    base_action_df = pd.concat([base_action_df,df_action],ignore_index=True)
    
    
    
        

Тестовая №1


In [429]:
df_mon

,гр.1,гр.2,гр.3,гр.4,гр.5,гр.6,гр.7,гр.8,гр.9,гр.10,...,гр.17,гр.18,гр.19,гр.20,гр.21,гр.22,гр.23,гр.24,гр.25,Название файла
0,Республика Бурятия,0323053578,"АО ""У-У ППО""",Потребность имеется,Профессии рабочих,Авербандщик,2,29.02.08 Технология обработки алмазов,10,6,...,1,1,1,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО,Тестовая №1
1,Республика Бурятия,0323053555,ООО Тест,Потребность отсутствует,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО,Тестовая №1
2,Республика Бурятия,0323053578,"АО ""У-У ППО""",Потребность имеется,Профессии рабочих,Сыросол,2,08.01.02 Монтажник трубопроводов,10,6,...,1,1,1,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО,Тестовая №1
3,Республика Бурятия,0323053578,"АО ""У-У ППО""",Потребность имеется,"Должности служащих, руководителей",Техник службы эксплуатации,2,55.02.01 Театральная и аудиовизуальная техника...,10,6,...,1,1,1,NaN,NaN,NaN,ВЕРНО,ВЕРНО,ВЕРНО,Тестовая №1


In [430]:
df_action

,гр.1,гр.2,гр.3,гр.4,гр.5,гр.6,гр.7,гр.8,гр.9,гр.10,гр.11,гр.12,Название файла
0,Республика Бурятия,0323053578,"АО ""У-У ППО""",мало платят,3,"больше платить,больше рекламы",проводятся мастер классы,1,30,6,0,ВЕРНО,Тестовая №1


In [431]:
inn_org_error_df = check_correct_relation(df_mon.copy(),2,3,4)
error_df = pd.concat([error_df, inn_org_error_df], axis=0, ignore_index=True)


In [432]:
t = time.localtime()
current_time = time.strftime('%H_%M_%S', t)
with pd.ExcelWriter(f'{path_to_end_folder}/Сводный файл ОПК от {current_time}.xlsx') as writer:
    base_mon_df.to_excel(writer,sheet_name='Форма по мониторингу',index=False)
    base_action_df.to_excel(writer,sheet_name='Форма по принимаемым мерам',index=False)

In [433]:
error_df

,Название файла,Строка или колонка с ошибкой,Описание ошибки


In [434]:
error_df.to_excel(f'{path_to_end_folder}/Ошибки от {current_time}.xlsx',index=False)